# AutoML in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a pipeline with Regression AutoML task.

**Motivations** - This notebook explains how to use Regression AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, command, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.automl import classification, regression
from azure.ai.ml.entities._job.automl.tabular import TabularFeaturizationSettings

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [3]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../../configuration.ipynb)

In [5]:
# Get a handle to workspace
ml_client = MLClient(
    credential=credential,
    workspace_name="ayushmishra_eastus2_ws",
    subscription_id="381b38e9-9840-4719-a5a0-61d9585e1e91",
    resource_group_name="ayush_mishra_res01",
)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/ayush_mishra_res01/providers/Microsoft.MachineLearningServices/workspaces/ayushmishra_eastus2_ws/computes/cpu-cluster', 'Resource__source_path': None, 'base_path': 'c:\\Users\\ayushmishra\\workspace\\azureml-examples\\sdk\\jobs\\pipelines\\1h_automl_in_pipeline\\automl-regression-house-pricing-in-pipeline', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001A7AAC25C00>, 'resource_id': None, 'location': 'eastus2', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 4, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': None, 'tier': 'dedicated'})


# 3. Basic pipeline job with regression task

## 3.1 Build pipeline

In [28]:
# Define pipeline
@pipeline(
    description="AutoML Regression Pipeline",
    )
def automl_regression(
    regression_train_data,
    regression_validation_data,
    regression_test_data
):
    command_func = command(
        inputs={"training_data": Input(type="mltable")},
        outputs={"train_data": Output(type="uri_folder")},
        command="python prepare_data.py --seed 250",
        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
    )

    copy_component = command_func(
        training_data = regression_train_data
    )

    copy_component.outputs.train_data.path = copy_component.outputs.train_data.path + "/MLTable"
    copy_component.outputs.train_data.type = "mltable"

    # define the AutoML regression task with AutoML function
    regression_node = regression(
        primary_metric="r2_score",
        target_column_name="SalePrice",
        training_data=copy_component.outputs.train_data,
        validation_data=regression_validation_data,
        test_data=regression_test_data,
        featurization=TabularFeaturizationSettings(mode="off"),
        # currently need to specify outputs "mlflow_model" explicitly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    # set limits & training
    regression_node.set_limits(max_trials=1, max_concurrent_trials=1)
    regression_node.set_training(enable_stack_ensemble=False, enable_vote_ensemble=False)

    # define command function for registering the model
    command_func = command(
        inputs=dict(
            model_input_path = Input(type="mlflow_model"),
            model_base_name= "regression_example_model"
        ),
        code="./register.py",
        command="python register.py " +
                "--model_input_path ${{inputs.model_input_path}} " +
                "--model_base_name ${{inputs.model_base_name}}",
        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
    )
    register_model = command_func(model_input_path = regression_node.outputs.best_model)


pipeline_regression = automl_regression(
    regression_train_data=Input(path="./training-mltable-folder/", type="mltable"),
    regression_validation_data=Input(path="./validation-mltable-folder/", type="mltable"),
    regression_test_data=Input(path="./test-mltable-folder/", type="mltable")
)

# set pipeline level compute
pipeline_regression.settings.default_compute="cpu-cluster"
        

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3671578676.py, line 13)

# 3.2 Submit pipeline job

In [22]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_regression, experiment_name="pipeline_samples"
)
pipeline_job

JobException: Supported input path value are ARM id, AzureML id, remote uri or local path.
Met <class 'azure.ai.ml._ml_exceptions.ValidationException'>:
No such file or directory: C:\Users\ayushmishra\workspace\azureml-examples\sdk\jobs\pipelines\1h_automl_in_pipeline\automl-regression-house-pricing-in-pipeline\{{copy_component.outputs.train_data}}\MLTable

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)